<a href="https://colab.research.google.com/github/zaaabik/hse/blob/master/application_dl/nlp_hw_1/HW_1_HLP_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Генерация текста

В первом домашнем задании мы с вами попробуем побыть писателями. Ваша задача построить алгоритм, который будет генерировать текст похожий на текст какого-либо известного писателя.

**Задание** 

N-gram model (3 балла)
* Препроцессинг текста с помощью NLTK (или любой подобной библиотеки)
* Разбиение train/test
* Построить модель на основе n-gram на train выборке
* Сгенерировать текст и посмотреть качество генерации на test выборки (perplexity)

GPT2 (5 баллов)
* Разбиение train/test
* Взять предобученый токенайзер gpt2
* Дообучить модель на основе gpt2 train выборка
* Сгенерировать текст и посмотреть качество генерации на test выборки (perplexity)
* Отправить ссылку на обученную модель в формате *ваш_ник*/gpt2-arxiv-clm


!! **Большинство кода уже написано в семинаре Language_modeling_solved.ipynb** !!

### Найти текст
Первый шаг для построения любой модели машинного обучения это поиск данных. Вам необходимо найти корпус текста писателя (на английском языке). Размер источника произвольный, но чем больше тем лучше. Желательно найти книгу или набор книг в текстовом формате.

In [ ]:
### Загрузка текста из источника

## N-gram language models 
По анологии с семинаром где мы пытались генерировать тексты научных статей мы сделаем предобработку текста. В зависимости от источника которым вы пользуетесь 
* Убираем мусор из текста
* Исключаем стоп слова
* Применяем токкенизатор 

In [2]:
lines = <todo> #List[str] лист строчек где каждый элемент является строкой

In [ ]:
tokenizer = <todo>
n_gramm_lines = # применить токенезатор

### Обучение
Далее обучаем n-gram модель для получения вероятностей цепочек слов 

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

In [ ]:
### Тут необходимо описать класс для подсчета вероятностей n-gram модели

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    <YOUR CODE>
    
    return counts


SyntaxError: ignored

In [ ]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)

In [ ]:
dummy_counts[('_UNK_', 'a')]

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        <YOUR CODE>
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
### Обучение n-gram модели

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    <YOUR CODE>

## Подсчет Perplexity

### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [ ]:
### Пример из семинара 
import numpy as np 
def perplexity(lm, lines, min_logprob=np.log(10 ** -7.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_length = 0
    log_pp = 0

    for line in tqdm(lines):
        tokens = [''] + line.split(' ') + [EOS]

        for t in range(1, len(tokens)):
            prefix = ' '.join(tokens[:t])
            log_pp += max(
                min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))
            )
            total_length += 1
    
    return np.exp(-( 1 / total_length) * log_pp)



In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(n_gramm_lines, test_size=0.25, random_state=42)

n = 2
lm = NGramLanguageModel(n=n, lines=train_lines)

ppx = perplexity(lm, test_lines)
print("N = %i, Perplexity = %.5f" % (n, ppx))


### Пример генерации текста

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    # <todo>
    
prefix = 'The long story short' # Придумайте первые несколько слов для вашего рассказа

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

Текст должен напоминать что-то осознанное

## Генерация с помощью нейронных сетей

По аналогии с семинаром, будем использова библиотеку transformers и предобученную модель gpt2.

In [ ]:
!pip install datasets transformers

В конце необходимо запушить модель на сайт huggingface, для этого необходимо получить токен
https://huggingface.co/docs/hub/security-tokens

In [ ]:
!huggingface-cli login --token <TODO>

In [ ]:
# Загружаем данные еще раз без какого-либо препроцессинга так как будем использовать готовый токенайзер

In [ ]:
clm_model_checkpoint = "gpt2"
clm_tokenizer_checkpoint = "gpt2"

from transformers import GPT2Tokenizer, AutoModelForCausalLM
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

Датасет можно можно не подвергать никакой обработке, так как в дальнейшем мы будем использовать предобученый токенайзер 
``` python
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
```

In [ ]:
lines = <todo> # Датасет тут можно не использовать никакой токенайзер так как в дальн

In [ ]:
train, valid = train_test_split(lines, test_size=0.2)
lm_datasets = {'train' : train, 'valid' : valid}

In [ ]:
### todo обучение модели подготовка датасетов для CausalLM
## Все есть в ноутбуке Language_modeling_solved

In [ ]:
training_args = TrainingArguments(
    f"gpt2-author-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['test'],
)

In [ ]:
trainer.train()

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
tokenizer.push_to_hub(
    'gpt2-author-clm'
)

### Генерация текста готовой моделью

In [ ]:
# сюда нужно вставить ваш ник из huggingface 
nick_name = <todo>

In [ ]:
from transformers import pipeline
generator = pipeline(
    'text-generation', 
    model = f'{nick_name}/gpt2-author-clm',
    tokenizer = tokenizer
)

In [ ]:
generator('The long story short')

После выполнения домашнего задания необходимо отправить ноутбук мне в телеграм: @zaaabik

Также туда выслать ссылку на модель в huggingface в формате: \*ваш_ник\*/gpt2-arxiv-clm